In [ ]:
from ultralytics import YOLO
import cv2

In [ ]:
# https://docs.ultralytics.com
model = YOLO("yolov8n.pt")

In [ ]:
from typing import NamedTuple


class Box(NamedTuple):
    x: float
    y: float
    w: float
    h: float
        
    @property
    def tuple(self):
        return self.x, self.y, self.w, self.h

        
class DetectedObject(NamedTuple):
    box: Box
    cls: int
    conf: int
        

def detect(model, image):
    objects = []
    for result in model.predict(image):
        for box in result.boxes:
            # https://docs.ultralytics.com/usage/python/#predict
            x, y, w, h = map(int, box.xywh[0])
            cls = int(box.cls)
            conf = int(float(box.conf) * 100)
            objects.append(DetectedObject(Box(x, y, w, h), cls, conf))
    return objects


def draw(image, objects):
    color = (255, 255, 0)
    count = 0
    for obj in objects:
        x, y, w, h = obj.box.tuple
        count += 1
        cv2.rectangle(image, (x - w // 2, y - h // 2), (x + w // 2, y + h // 2), color, 2)
        cv2.putText(image, f'{obj.cls}: {obj.conf}%', (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    cv2.putText(image, f'Count: {count}', (40, 70), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 2)

In [ ]:
from IPython.display import display, Image

def image_from_camera():
    video = cv2.VideoCapture(0)
    _, image = video.read()
    video.release()
    return image

def image_from_file(filepath):
    return cv2.imread(filepath)

def show(image):
    window = display(None, display_id=True)
    _, image = cv2.imencode('.jpeg', image)
    window.update(Image(data = image.tobytes()))

In [ ]:
image = image_from_file('res/medium-people-5-dark.jpg')
boxes = detect(model, image)
draw(image, boxes)
show(image)